In [36]:
# Import the Python library and module and configure running information.
import math
import numpy as np
import pandas as pd
import os
import math
import random
import codecs
from pathlib import Path
import mindspore
import mindspore.dataset as ds
import mindspore.nn as nn
from mindspore import Tensor
from mindspore import context
from mindspore.train.model import Model
from mindspore.nn.metrics import Accuracy
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor, TimeMonitor
from mindspore.ops import operations as ops
from easydict import EasyDict as edict
cfg = edict({
 'name': 'movie review',
 'pre_trained': False,
 'num_classes': 2,
 'batch_size': 64,
 'epoch_size': 4,
 'weight_decay': 3e-5,
 'data_path': 'C:/Users/DELL/Documents/HCIA/HCIA-AI V3.5/X-Experiment-Guide/2. Deep Learning and AI Development Framework Lab Guide/TextCNN/data/',
 'device_target': 'CPU',
 'device_id': 0,
 'keep_checkpoint_max': 1,
 'checkpoint_path': './ckpt/train_textcnn-4_149.ckpt',
 'word_len': 51,
 'vec_length': 40
})
mindspore.set_device(cfg.device_target)
context.set_context(mode=context.GRAPH_MODE)

In [37]:
# Read and process data.
with open("C:/Users/DELL/Documents/HCIA/HCIA-AI V3.5/X-Experiment-Guide/2. Deep Learning and AI Development Framework Lab Guide/TextCNN/data/rt-polarity.neg", 'r', encoding='utf-8') as f:
 print("Negative reivews:")
 for i in range(5):
     print("[{0}]:{1}".format(i,f.readline()))
with open("C:/Users/DELL/Documents/HCIA/HCIA-AI V3.5/X-Experiment-Guide/2. Deep Learning and AI Development Framework Lab Guide/TextCNN/data/rt-polarity.pos", 'r', encoding='utf-8') as f:
 print("Positive reivews:")
 for i in range(5):
     print("[{0}]:{1}".format(i,f.readline()))

Negative reivews:
[0]:simplistic , silly and tedious . 

[1]:it's so laddish and juvenile , only teenage boys could possibly find it funny . 

[2]:exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable . 

[3]:[garbus] discards the potential for pathological study , exhuming instead , the skewed melodrama of the circumstantial situation . 

[4]:a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification . 

Positive reivews:
[0]:the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 

[1]:the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth . 

[2]:effective but too-tepid biopic

In [38]:
# Define the data generation class.
class Generator():
 def __init__(self, input_list):
     self.input_list=input_list
 def __getitem__(self,item):
     return (np.array(self.input_list[item][0],dtype=np.int32), np.array(self.input_list[item][1],dtype=np.int32))
 def __len__(self):
     return len(self.input_list)
    
class MovieReview:
 '''
 Movie review dataset
 '''
 def __init__(self, root_dir, maxlen, split):
     '''
     input:
         root_dir: movie review data directory
         maxlen: maximum length of a sentence
         split: the training/validation ratio in the dataset
     '''
     self.path = root_dir
     self.feelMap = {
         'neg':0,
         'pos':1
         }
     self.files = []
     self.doConvert = False

     mypath = Path(self.path)
     if not mypath.exists() or not mypath.is_dir():
         print("please check the root_dir!")
         raise ValueError
     # Find the files in the data directory.
     for root,_,filename in os.walk(self.path):
         for each in filename:
             self.files.append(os.path.join(root,each))
         break
     # Check whether there are two files: .neg and .pos.
     if len(self.files) != 2:
         print("There are {} files in the root_dir".format(len(self.files)))
         raise ValueError
            
     # Read data.
     self.word_num = 0
     self.maxlen = 0
     self.minlen = float("inf")
     self.maxlen = float("-inf")
     self.Pos = []
     self.Neg = []
     for filename in self.files:
         self.read_data(filename)
     self.text2vec(maxlen=maxlen)
     self.split_dataset(split=split)
 
 def read_data(self, filePath):
     with open(filePath,'r') as f:
         for sentence in f.readlines():
             sentence = sentence.replace('\n', '')\
                               .replace('"', '')\
                               .replace('\'', '')\
                               .replace('.', '')\
                               .replace(',', '')\
                               .replace('[', '')\
                               .replace(']', '')\
                               .replace('(', '')\
                               .replace(')', '')\
                               .replace(':', '')\
                               .replace('--', '')\
                               .replace('-', ' ')\
                               .replace('\\', '')\
                               .replace('0', '')\
                               .replace('1', '')\
                               .replace('2', '')\
                               .replace('3', '')\
                               .replace('4', '')\
                               .replace('5', '')\
                               .replace('6', '')\
                               .replace('7', '')\
                               .replace('8', '')\
                               .replace('9', '')\
                               .replace('`', '')\
                               .replace('=', '')\
                               .replace('$', '')\
                               .replace('/', '')\
                               .replace('*', '')\
                               .replace(';', '')\
                               .replace('<b>', '')\
                               .replace('%', '')
            
             sentence = sentence.split(' ')
             sentence = list(filter(lambda x: x, sentence))
             if sentence:
                 self.word_num += len(sentence)
                 self.maxlen = self.maxlen if self.maxlen >= len(sentence) else len(sentence)
                 self.minlen = self.minlen if self.minlen <= len(sentence) else len(sentence)
                 if 'pos' in filePath:
                     self.Pos.append([sentence,self.feelMap['pos']])
                 else:
                     self.Neg.append([sentence,self.feelMap['neg']])
                        

 def text2vec(self, maxlen):
     '''
        # Convert sentences into vectors.
     '''
     # Vocab = {word : index}
     self.Vocab = dict()
     # self.Vocab['None']
     for SentenceLabel in self.Pos+self.Neg:
         vector = [0]*maxlen
         for index, word in enumerate(SentenceLabel[0]):
             if index >= maxlen:
                 break
             if word not in self.Vocab.keys():
                 self.Vocab[word] = len(self.Vocab)
                 vector[index] = len(self.Vocab) - 1
             else:
                 vector[index] = self.Vocab[word]
         SentenceLabel[0] = vector
         self.doConvert = True
            
            
 def split_dataset(self, split):
     '''
     Divide the dataset into a training set and a test set.
     '''
     trunk_pos_size = math.ceil((1-split)*len(self.Pos))
     trunk_neg_size = math.ceil((1-split)*len(self.Neg))
     trunk_num = int(1/(1-split))
     pos_temp=list()
     neg_temp=list()
     for index in range(trunk_num):
         pos_temp.append(self.Pos[index*trunk_pos_size:(index+1)*trunk_pos_size])
         neg_temp.append(self.Neg[index*trunk_neg_size:(index+1)*trunk_neg_size])
     self.test = pos_temp.pop(2)+neg_temp.pop(2)
     self.train = [i for item in pos_temp+neg_temp for i in item]
     random.shuffle(self.train)


 def get_dict_len(self):
     '''
     Obtain the length of a dictionary consisting of characters in a dataset.
     '''
     if self.doConvert:
         return len(self.Vocab)
     else:
         print("Haven't finished Text2Vec")
         return -1
        
 def create_train_dataset(self, epoch_size, batch_size):
     dataset = ds.GeneratorDataset(source=Generator(input_list=self.train), column_names=["data","label"], shuffle=False)
     dataset=dataset.batch(batch_size=batch_size,drop_remainder=True)
     dataset=dataset.repeat(epoch_size)
     return dataset

 def create_test_dataset(self, batch_size):
     dataset = ds.GeneratorDataset(source=Generator(input_list=self.test), column_names=["data","label"], shuffle=False)
     dataset=dataset.batch(batch_size=batch_size,drop_remainder=True)
     return dataset

In [39]:
instance = MovieReview(root_dir=cfg.data_path, maxlen=cfg.word_len, split=0.9)
dataset = instance.create_train_dataset(batch_size=cfg.batch_size,epoch_size=cfg.epoch_size)
batch_num = dataset.get_dataset_size()

In [40]:
# Code for displaying the data processing result:
vocab_size=instance.get_dict_len()
print("vocab_size:{0}".format(vocab_size))
item =dataset.create_dict_iterator()
for i,data in enumerate(item):
 if i<1:
     print(data)
     print(data['data'][1])
 else:
     break

vocab_size:18848
{'data': Tensor(shape=[64, 51], dtype=Int32, value=
[[ 128, 2727,  122 ...    0,    0,    0],
 [ 726,    0,  198 ...    0,    0,    0],
 [7777,    5, 3466 ...    0,    0,    0],
 ...
 [  15, 5149, 5555 ...    0,    0,    0],
 [ 345, 2784, 3073 ...    0,    0,    0],
 [4773, 4774,    2 ...    0,    0,    0]]), 'label': Tensor(shape=[64], dtype=Int32, value= [0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 
 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 
 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1])}
[  726     0   198  1716   111  5104    15  1457  1300   247  2577   655
    64    10  3494   710  1793 11565   415    82   118  1609  3635 11566
  1847   855  1177    10    15  1519   694    32  1442     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0]


In [43]:
#Configure training parameters.
#Code for setting the learning rate:
learning_rate = []
warm_up = [1e-3 / math.floor(cfg.epoch_size / 5) * (i + 1) for _ in range(batch_num)
 for i in range(math.floor(cfg.epoch_size / 5))]
shrink = [1e-3 / (16 * (i + 1)) for _ in range(batch_num)
 for i in range(math.floor(cfg.epoch_size * 3 / 5))]
normal_run = [1e-3 for _ in range(batch_num) for i in
 range(cfg.epoch_size - math.floor(cfg.epoch_size / 5)
 - math.floor(cfg.epoch_size * 2 / 5))]
learning_rate = learning_rate + warm_up + normal_run + shrink

In [45]:
# Define a TextCNN mode
def _weight_variable(shape, factor=0.01):
 init_value = np.random.randn(*shape).astype(np.float32) * factor
 return Tensor(init_value)
def make_conv_layer(kernel_size):
 weight_shape = (96, 1, *kernel_size)
 weight = _weight_variable(weight_shape)
 return nn.Conv2d(in_channels=1, out_channels=96, kernel_size=kernel_size, padding=1, pad_mode="pad", weight_init=weight, has_bias=True)

class TextCNN(nn.Cell):
 def __init__(self, vocab_len, word_len, num_classes, vec_length):
     super(TextCNN, self).__init__()
     self.vec_length = vec_length
     self.word_len = word_len
     self.num_classes = num_classes
     self.unsqueeze = ops.ExpandDims()
     self.embedding = nn.Embedding(vocab_len, self.vec_length, embedding_table='normal')
     self.slice = ops.Slice()
     self.layer1 = self.make_layer(kernel_height=3)
     self.layer2 = self.make_layer(kernel_height=4)
     self.layer3 = self.make_layer(kernel_height=5)
     self.concat = ops.Concat(1)
     self.fc = nn.Dense(96*3, self.num_classes)
     self.drop = nn.Dropout(p=0.5)
     self.print = ops.Print()
     self.reducemean = ops.ReduceMax(keep_dims=False)

 def make_layer(self, kernel_height):
     return nn.SequentialCell(
         [
             make_conv_layer((kernel_height,self.vec_length)),
             nn.ReLU(),
             nn.MaxPool2d(kernel_size=(self.word_len-kernel_height+1,1)),
         ])
    
 def construct(self,x):
     x = self.unsqueeze(x, 1)
     x = self.embedding(x)
     x1 = self.layer1(x)
     x2 = self.layer2(x)
     x3 = self.layer3(x)
     x1 = self.reducemean(x1, (2, 3))
     x2 = self.reducemean(x2, (2, 3))
     x3 = self.reducemean(x3, (2, 3))
     x = self.concat((x1, x2, x3))
     x = self.drop(x)
     x = self.fc(x)
     return x

net = TextCNN(vocab_len=instance.get_dict_len(), word_len=cfg.word_len, num_classes=cfg.num_classes, vec_length=cfg.vec_length)
print(net)


TextCNN<
  (embedding): Embedding<vocab_size=18848, embedding_size=40, use_one_hot=False, embedding_table=Parameter (name=embedding.embedding_table, shape=(18848, 40), dtype=Float32, requires_grad=True), dtype=Float32, padding_idx=None>
  (layer1): SequentialCell<
    (0): Conv2d<input_channels=1, output_channels=96, kernel_size=(3, 40), stride=(1, 1), pad_mode=pad, padding=1, dilation=(1, 1), group=1, has_bias=True, weight_init=[[[[ 0.00027819 -0.00079187  0.01734651 ... -0.000179   -0.00204298
         0.01453898]
       [ 0.0107973   0.00694471  0.00698016 ...  0.00153196 -0.00752804
        -0.00645664]
       [-0.01075113  0.01450333  0.00043253 ...  0.00591194 -0.00488818
        -0.00259404]]]
    
    
     [[[ 0.00917239  0.00591114 -0.00731728 ...  0.00213329 -0.00162149
        -0.01235505]
       [-0.00463171  0.00391021 -0.00767997 ... -0.00693404 -0.02037086
         0.01444044]
       [ 0.00574569  0.00505107  0.00224499 ...  0.0050235  -0.00736969
        -0.02392101]]]

In [47]:
# Define training parameters.
# Optimizer, loss function, checkpoint, and time monitor settings
opt = nn.Adam(filter(lambda x: x.requires_grad, net.get_parameters()), learning_rate=learning_rate, weight_decay=cfg.weight_decay)
loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True)
model = Model(net, loss_fn=loss, optimizer=opt, metrics={'acc': Accuracy()})
config_ck = CheckpointConfig(save_checkpoint_steps=int(cfg.epoch_size*batch_num/2), keep_checkpoint_max=cfg.keep_checkpoint_max)
time_cb = TimeMonitor(data_size=batch_num)
ckpt_save_dir = "./ckpt"
ckpoint_cb = ModelCheckpoint(prefix="train_textcnn", directory=ckpt_save_dir, config=config_ck)
loss_cb = LossMonitor()

In [49]:
# Start training.
model.train(cfg.epoch_size, dataset, callbacks=[time_cb, ckpoint_cb, loss_cb])
print("train success")

epoch: 1 step: 1, loss is 0.6973013281822205
epoch: 1 step: 2, loss is 0.695469081401825
epoch: 1 step: 3, loss is 0.6898638010025024
epoch: 1 step: 4, loss is 0.6924630403518677
epoch: 1 step: 5, loss is 0.693630576133728
epoch: 1 step: 6, loss is 0.6910443305969238
epoch: 1 step: 7, loss is 0.6881700158119202
epoch: 1 step: 8, loss is 0.6828582286834717
epoch: 1 step: 9, loss is 0.6851547956466675
epoch: 1 step: 10, loss is 0.6964682340621948
epoch: 1 step: 11, loss is 0.6865369081497192
epoch: 1 step: 12, loss is 0.6884020566940308
epoch: 1 step: 13, loss is 0.6969497203826904
epoch: 1 step: 14, loss is 0.6922197937965393
epoch: 1 step: 15, loss is 0.6972231864929199
epoch: 1 step: 16, loss is 0.6979447603225708
epoch: 1 step: 17, loss is 0.6974633932113647
epoch: 1 step: 18, loss is 0.709978461265564
epoch: 1 step: 19, loss is 0.6908731460571289
epoch: 1 step: 20, loss is 0.694514274597168
epoch: 1 step: 21, loss is 0.6897791624069214
epoch: 1 step: 22, loss is 0.6855040788650513
e

epoch: 1 step: 179, loss is 0.5115837454795837
epoch: 1 step: 180, loss is 0.5648525357246399
epoch: 1 step: 181, loss is 0.4693623185157776
epoch: 1 step: 182, loss is 0.4225767254829407
epoch: 1 step: 183, loss is 0.4969969093799591
epoch: 1 step: 184, loss is 0.40389835834503174
epoch: 1 step: 185, loss is 0.3793712854385376
epoch: 1 step: 186, loss is 0.48216065764427185
epoch: 1 step: 187, loss is 0.5568665266036987
epoch: 1 step: 188, loss is 0.5933369994163513
epoch: 1 step: 189, loss is 0.4893414080142975
epoch: 1 step: 190, loss is 0.4058995246887207
epoch: 1 step: 191, loss is 0.5018218755722046
epoch: 1 step: 192, loss is 0.42437395453453064
epoch: 1 step: 193, loss is 0.5277137756347656
epoch: 1 step: 194, loss is 0.4961119592189789
epoch: 1 step: 195, loss is 0.4043952226638794
epoch: 1 step: 196, loss is 0.44009166955947876
epoch: 1 step: 197, loss is 0.3893536925315857
epoch: 1 step: 198, loss is 0.5026965141296387
epoch: 1 step: 199, loss is 0.5182700157165527
epoch: 1 

epoch: 1 step: 352, loss is 0.19517117738723755
epoch: 1 step: 353, loss is 0.22623595595359802
epoch: 1 step: 354, loss is 0.24427101016044617
epoch: 1 step: 355, loss is 0.25317585468292236
epoch: 1 step: 356, loss is 0.19535784423351288
epoch: 1 step: 357, loss is 0.2565336227416992
epoch: 1 step: 358, loss is 0.21937289834022522
epoch: 1 step: 359, loss is 0.35364192724227905
epoch: 1 step: 360, loss is 0.25799888372421265
epoch: 1 step: 361, loss is 0.2114674299955368
epoch: 1 step: 362, loss is 0.20559744536876678
epoch: 1 step: 363, loss is 0.33955225348472595
epoch: 1 step: 364, loss is 0.34074538946151733
epoch: 1 step: 365, loss is 0.23339179158210754
epoch: 1 step: 366, loss is 0.2507880926132202
epoch: 1 step: 367, loss is 0.28166836500167847
epoch: 1 step: 368, loss is 0.3011850416660309
epoch: 1 step: 369, loss is 0.15973493456840515
epoch: 1 step: 370, loss is 0.22431302070617676
epoch: 1 step: 371, loss is 0.3301381766796112
epoch: 1 step: 372, loss is 0.285782396793365

epoch: 1 step: 524, loss is 0.0561862550675869
epoch: 1 step: 525, loss is 0.16027170419692993
epoch: 1 step: 526, loss is 0.0683010071516037
epoch: 1 step: 527, loss is 0.1729481816291809
epoch: 1 step: 528, loss is 0.10944504290819168
epoch: 1 step: 529, loss is 0.19449129700660706
epoch: 1 step: 530, loss is 0.09506841003894806
epoch: 1 step: 531, loss is 0.1252337545156479
epoch: 1 step: 532, loss is 0.04992973804473877
epoch: 1 step: 533, loss is 0.05496437847614288
epoch: 1 step: 534, loss is 0.0705384910106659
epoch: 1 step: 535, loss is 0.10230949521064758
epoch: 1 step: 536, loss is 0.1396767646074295
epoch: 1 step: 537, loss is 0.1417720764875412
epoch: 1 step: 538, loss is 0.06444476544857025
epoch: 1 step: 539, loss is 0.0993741899728775
epoch: 1 step: 540, loss is 0.09792280197143555
epoch: 1 step: 541, loss is 0.07745826989412308
epoch: 1 step: 542, loss is 0.054382458329200745
epoch: 1 step: 543, loss is 0.07800719141960144
epoch: 1 step: 544, loss is 0.13376069068908691

epoch: 2 step: 100, loss is 0.025838086381554604
epoch: 2 step: 101, loss is 0.024708829820156097
epoch: 2 step: 102, loss is 0.021644342690706253
epoch: 2 step: 103, loss is 0.02983747236430645
epoch: 2 step: 104, loss is 0.0430714450776577
epoch: 2 step: 105, loss is 0.04728925973176956
epoch: 2 step: 106, loss is 0.11792132258415222
epoch: 2 step: 107, loss is 0.04197564721107483
epoch: 2 step: 108, loss is 0.08654165267944336
epoch: 2 step: 109, loss is 0.03271026909351349
epoch: 2 step: 110, loss is 0.09239818900823593
epoch: 2 step: 111, loss is 0.03798660635948181
epoch: 2 step: 112, loss is 0.014649596996605396
epoch: 2 step: 113, loss is 0.010762056335806847
epoch: 2 step: 114, loss is 0.04231097549200058
epoch: 2 step: 115, loss is 0.022679384797811508
epoch: 2 step: 116, loss is 0.02379770018160343
epoch: 2 step: 117, loss is 0.034982699900865555
epoch: 2 step: 118, loss is 0.03810197487473488
epoch: 2 step: 119, loss is 0.038688816130161285
epoch: 2 step: 120, loss is 0.031

...

epoch: 3 step: 508, loss is 0.0027318489737808704
epoch: 3 step: 509, loss is 0.0014585313620045781
epoch: 3 step: 510, loss is 0.0005975640378892422
epoch: 3 step: 511, loss is 0.002078673802316189
epoch: 3 step: 512, loss is 0.002552863210439682
epoch: 3 step: 513, loss is 0.002011217875406146
epoch: 3 step: 514, loss is 0.0012101861648261547
epoch: 3 step: 515, loss is 0.027787478640675545
epoch: 3 step: 516, loss is 0.007723640184849501
epoch: 3 step: 517, loss is 0.0008014862542040646
epoch: 3 step: 518, loss is 0.0006870978395454586
epoch: 3 step: 519, loss is 0.0013966740807518363
epoch: 3 step: 520, loss is 0.005327657330781221
epoch: 3 step: 521, loss is 0.0007668447215110064
epoch: 3 step: 522, loss is 0.0009921382879838347
epoch: 3 step: 523, loss is 0.0009937757859006524
epoch: 3 step: 524, loss is 0.001851545413956046
epoch: 3 step: 525, loss is 0.004544113762676716
epoch: 3 step: 526, loss is 0.000862718210555613
epoch: 3 step: 527, loss is 0.00107235973700881
epoch: 3 st

epoch: 4 step: 78, loss is 0.0005862310063093901
epoch: 4 step: 79, loss is 0.0008188940701074898
epoch: 4 step: 80, loss is 0.0006333521450869739
epoch: 4 step: 81, loss is 0.0014927085721865296
epoch: 4 step: 82, loss is 0.001058823661878705
epoch: 4 step: 83, loss is 0.0005476605147123337
epoch: 4 step: 84, loss is 0.0007796216523274779
epoch: 4 step: 85, loss is 0.0016875698929652572
epoch: 4 step: 86, loss is 0.005690303631126881
epoch: 4 step: 87, loss is 0.00018843142606783658
epoch: 4 step: 88, loss is 0.0006588185206055641
epoch: 4 step: 89, loss is 0.0004437825991772115
epoch: 4 step: 90, loss is 0.0016883539501577616
epoch: 4 step: 91, loss is 0.0025844324845820665
epoch: 4 step: 92, loss is 0.0010033224243670702
epoch: 4 step: 93, loss is 0.0008838747162371874
epoch: 4 step: 94, loss is 0.000800902140326798
epoch: 4 step: 95, loss is 0.00032166732125915587
epoch: 4 step: 96, loss is 0.0007417488377541304
epoch: 4 step: 97, loss is 0.0019153701141476631
epoch: 4 step: 98, lo

epoch: 4 step: 243, loss is 0.00035363645292818546
epoch: 4 step: 244, loss is 0.00043834804091602564
epoch: 4 step: 245, loss is 0.0005638175643980503
epoch: 4 step: 246, loss is 0.0021363685373216867
epoch: 4 step: 247, loss is 0.0003540718462318182
epoch: 4 step: 248, loss is 0.0006291775498539209
epoch: 4 step: 249, loss is 0.0005637489375658333
epoch: 4 step: 250, loss is 0.0006868296768516302
epoch: 4 step: 251, loss is 0.000555549340788275
epoch: 4 step: 252, loss is 0.0017904802225530148
epoch: 4 step: 253, loss is 0.0009321554680354893
epoch: 4 step: 254, loss is 0.0006437670672312379
epoch: 4 step: 255, loss is 0.0006579551263712347
epoch: 4 step: 256, loss is 0.00014764275692868978
epoch: 4 step: 257, loss is 0.0005463006673380733
epoch: 4 step: 258, loss is 0.0008140108548104763
epoch: 4 step: 259, loss is 0.0011077006347477436
epoch: 4 step: 260, loss is 0.0003425783070269972
epoch: 4 step: 261, loss is 0.00029060401720926166
epoch: 4 step: 262, loss is 0.00066509761381894

epoch: 4 step: 407, loss is 0.000577404978685081
epoch: 4 step: 408, loss is 0.0007110236911103129
epoch: 4 step: 409, loss is 0.002597525017336011
epoch: 4 step: 410, loss is 0.0016517096664756536
epoch: 4 step: 411, loss is 0.00016827831859700382
epoch: 4 step: 412, loss is 0.0005503942957147956
epoch: 4 step: 413, loss is 0.0005697251763194799
epoch: 4 step: 414, loss is 0.0007295742980204523
epoch: 4 step: 415, loss is 0.0005466490983963013
epoch: 4 step: 416, loss is 0.0026768811512738466
epoch: 4 step: 417, loss is 0.0002832973259501159
epoch: 4 step: 418, loss is 0.00123522337526083
epoch: 4 step: 419, loss is 0.0006161880446597934
epoch: 4 step: 420, loss is 0.0009034522227011621
epoch: 4 step: 421, loss is 0.0005314509035088122
epoch: 4 step: 422, loss is 0.0009445046889595687
epoch: 4 step: 423, loss is 0.0003889875370077789
epoch: 4 step: 424, loss is 0.0026963362470269203
epoch: 4 step: 425, loss is 0.0017353224102407694
epoch: 4 step: 426, loss is 0.0017053545452654362
epo

epoch: 4 step: 571, loss is 0.0005751006538048387
epoch: 4 step: 572, loss is 0.00033876363886520267
epoch: 4 step: 573, loss is 0.0006561881164088845
epoch: 4 step: 574, loss is 0.006803364958614111
epoch: 4 step: 575, loss is 0.0008383302483707666
epoch: 4 step: 576, loss is 0.00020334456348791718
epoch: 4 step: 577, loss is 0.000977130839601159
epoch: 4 step: 578, loss is 0.0008503232384100556
epoch: 4 step: 579, loss is 0.00026175682432949543
epoch: 4 step: 580, loss is 0.0007281180587597191
epoch: 4 step: 581, loss is 0.0007681372808292508
epoch: 4 step: 582, loss is 0.0005760818021371961
epoch: 4 step: 583, loss is 0.0005073759239166975
epoch: 4 step: 584, loss is 0.00034306757152080536
epoch: 4 step: 585, loss is 0.00040538483881391585
epoch: 4 step: 586, loss is 0.0002640231396071613
epoch: 4 step: 587, loss is 0.00041906567639671266
epoch: 4 step: 588, loss is 0.0013691240455955267
epoch: 4 step: 589, loss is 0.0011165792820975184
epoch: 4 step: 590, loss is 0.0021648453548550

In [50]:
def preprocess(sentence):
 sentence = sentence.lower().strip()
 sentence = sentence.replace('\n','')\
        .replace('"','')\
        .replace('\'','')\
        .replace('.','')\
        .replace(',','')\
        .replace('[','')\
        .replace(']','')\
        .replace('(','')\
        .replace(')','')\
        .replace(':','')\
        .replace('--','')\
        .replace('-',' ')\
        .replace('\\','')\
        .replace('0','')\
        .replace('1','')\
        .replace('2','')\
        .replace('3','')\
        .replace('4','')\
        .replace('5','')\
        .replace('6','')\
        .replace('7','')\
        .replace('8','')\
        .replace('9','')\
        .replace('`','')\
        .replace('=','')\
        .replace('$','')\
        .replace('/','')\
        .replace('*','')\
        .replace(';','')\
        .replace('<b>','')\
        .replace('%','')\
        .replace(" "," ")
 sentence = sentence.split(' ')
 maxlen = cfg.word_len
 vector = [0]*maxlen
 for index, word in enumerate(sentence):
     if index >= maxlen:
         break
     if word not in instance.Vocab.keys():
         print(word,"The word does not appear in the dictionary.")
     else:
         vector[index] = instance.Vocab[word]
 sentence = vector
 return sentence

def inference(review_en):
 review_en = preprocess(review_en)
 input_en = Tensor(np.array([review_en]).astype(np.int32))
 output = net(input_en)
 if np.argmax(np.array(output[0])) == 1:
     print("Positive comments")
 else:
     print("Negative comments")


In [51]:
review_en = "the movie is so boring"
inference(review_en)

Negative comments
